In [4]:
import librosa
import soundfile as sf
import pandas as pd
import copy
from tqdm import tqdm
import numpy as np
import random

# dataset = pd.read_csv("dataset.csv", sep=";")
# dataset = dataset[dataset["lang"]=="eng"]
# dataset["augmentation"] = ""
# dataset.to_csv("dataset_eng.csv", sep=";", index=False)

In [10]:
import numpy as np
import pandas as pd
dataset = pd.read_csv("dataset_eng.csv", sep=";")

def create_augmentation_slow_down(path: str, language: str):
    y, sr = librosa.load(f"../{path}")
    new_file_name = path[:-4].split(f"/{language}")[1]
    y_slow = librosa.effects.time_stretch(y, rate=random.uniform(0.4, 0.6))
    new_path = f"../StarGANv2-EmotionalVC/dataset/{language}_syn{new_file_name}_aug_slowed.wav"
    sf.write(f"../../StarGANv2-EmotionalVC/dataset/{language}_syn{new_file_name}_aug_slowed.wav", y_slow, sr)
    return new_path

def create_augmetation_pitch_shift_up(path: str, language: str):
    y, sr = librosa.load(f"../{path}")
    new_file_name = path[:-4].split(f"/{language}")[1]
    y_shift = librosa.effects.pitch_shift(y, sr=sr, n_steps=4)
    new_path = f"../StarGANv2-EmotionalVC/dataset/{language}_syn{new_file_name}_aug_pitch.wav"
    sf.write(f"../../StarGANv2-EmotionalVC/dataset/{language}_syn{new_file_name}_aug_pitch.wav", y_shift, sr)
    return new_path

def create_augmetation_pitch_shift_down(path: str, language: str):
    y, sr = librosa.load(f"../{path}")
    new_file_name = path[:-4].split(f"/{language}")[1]
    y_shift = librosa.effects.pitch_shift(y, sr=sr, n_steps=-4)
    new_path = f"../StarGANv2-EmotionalVC/dataset/{language}_syn{new_file_name}_aug_pitch_2.wav"
    sf.write(f"../../StarGANv2-EmotionalVC/dataset/{language}_syn{new_file_name}_aug_pitch_2.wav", y_shift, sr)
    return new_path

def create_augmetation_noise(path, language, noise_factor=0.2):
    y, sr = librosa.load(f"../{path}")
    new_file_name = path[:-4].split(f"/{language}")[1]
    noise = np.random.randn(len(y)) * random.uniform(0.01, 0.2)
    new_path = f"../StarGANv2-EmotionalVC/dataset/{language}_syn{new_file_name}_aug_noise.wav"
    sf.write(f"../../StarGANv2-EmotionalVC/dataset/{language}_syn{new_file_name}_aug_noise.wav", y+noise, sr)
    return new_path

def create_augmetation_volume_scale(path, language, scale_factor=0.8):
    y, sr = librosa.load(f"../{path}")
    new_file_name = path[:-4].split(f"/{language}")[1]
    y_scale = y * scale_factor
    new_path = f"../StarGANv2-EmotionalVC/dataset/{language}_syn{new_file_name}_aug_volume_scale.wav"
    sf.write(f"../../StarGANv2-EmotionalVC/dataset/{language}_syn{new_file_name}_aug_volume_scale.wav", y_scale, sr)
    return new_path

In [11]:

for index, row in tqdm(dataset.iterrows(), total=dataset.shape[0]):
    _row_slow = copy.deepcopy(row)
    _row_slow["path"] = create_augmentation_slow_down(_row_slow["path"], _row_slow["lang"])
    _row_slow["augmentation"] = "slow_down"
    
    _row_pitch = copy.deepcopy(row)
    _row_pitch["path"] = create_augmetation_pitch_shift_up(_row_pitch["path"], _row_pitch["lang"])
    _row_pitch["augmentation"] = "pitch_shift"
    
    _row_pitch_2 = copy.deepcopy(row)
    _row_pitch_2["path"] = create_augmetation_pitch_shift_down(_row_pitch_2["path"], _row_pitch_2["lang"])
    _row_pitch_2["augmentation"] = "pitch_shift_2"
    
    _row_noise = copy.deepcopy(row)
    _row_noise["path"] = create_augmetation_noise(_row_noise["path"], _row_noise["lang"])
    _row_noise["augmentation"] = "noise"
    
    dataset = dataset.append([_row_pitch, _row_noise, _row_pitch, _row_pitch_2, _row_slow])
    
    
    


  0%|          | 0/14202 [00:00<?, ?it/s]/tmp/ipykernel_8892/1811476441.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append([_row_pitch, _row_noise, _row_pitch, _row_pitch_2, _row_slow])
  0%|          | 1/14202 [00:01<5:33:36,  1.41s/it]/tmp/ipykernel_8892/1811476441.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append([_row_pitch, _row_noise, _row_pitch, _row_pitch_2, _row_slow])
  0%|          | 2/14202 [00:02<4:22:45,  1.11s/it]/tmp/ipykernel_8892/1811476441.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append([_row_pitch, _row_noise, _row_pitch, _row_pitch_2, _row_slow])
  0%|          | 3/14202 [00:03<4:41:02,  1.19s/it]/tmp/ipykernel_8892/181

In [1]:
dataset.reset_index().to_csv("./dataset_eng_with_aug.csv",sep=";",index=False,columns=["dataset","lang","path","actor_id","gender","emotion","statement_id","augmentation"])

NameError: name 'dataset' is not defined

In [ ]:
dataset_aug=dataset_aug.reset_index()

In [4]:
dataset_aug = pd.read_csv("dataset_with_aug.csv", sep=";")
dataset_happy = dataset_aug[(dataset_aug["emotion"]=="happy") | (dataset_aug["emotion"]=="neutral")]

preparing_dataset=[]
for index, group in dataset_happy.groupby(["dataset","actor_id","statement_id"]):
    try:
        neutral = group[group["emotion"] == "neutral"].iloc[0]
    except Exception as ex:
        continue
    for _nested_index, _nested_row in group[group["emotion"] != "neutral"].iterrows():
        if neutral["dataset"]!=_nested_row["dataset"]:
            print("")
        preparing_dataset.append([neutral["path"], _nested_row["path"]])

preparing_dataset = pd.DataFrame(preparing_dataset).sample(frac=1)

In [6]:
import yaml

################################################
config_path="../Configs/config.yml"
config = yaml.safe_load(open(config_path))

training_set_percentage = config.get('training_set_percentage', 80)
validation_set_percentage = config.get('validation_set_percentage', 20)

training_path = config.get('training_path', "../Data/eng_aug_happy_training_list.txt")
validation_path = config.get('validation_path', "../Data/eng_aug_happy_validation_list.txt" )
################################################
preparing_dataset = pd.DataFrame(preparing_dataset).sample(frac=1)
# Define stream 
# pick sample for training and validation
training_dataframe = preparing_dataset.iloc[0:int((preparing_dataset.shape[0]*training_set_percentage)/100)]
validation_dataframe = preparing_dataset.iloc[preparing_dataset.shape[0]-int((preparing_dataset.shape[0]*validation_set_percentage)/100):]

training_file = open(training_path,"w")
# Create training file
for index, row in training_dataframe.iterrows():
    try:
        training_file.write(f"{row[0]}|{row[1]}\n")
    except IOError as e:
        print(e)
training_file.close()


validation_file = open(validation_path,"w")
# Create validation file
for index, row in validation_dataframe.iterrows():
    try:
        validation_file.write(f"{row[0]}|{row[1]}\n")
    except IOError as e:
        print(e)
validation_file.close()